# <center>**Trabajo Práctico Número 1**</center>
### <center>**Algoritmos de búsqueda en el problema de la Torre de Hanoi**</center>
#### <center>Introducción a la Inteligencia Artificial - CEIA & DPLN, Fiuba</center>

<center><img width="35%" src="https://lh5.googleusercontent.com/cPAoMij8-mRhQEFe6cf2R1bncilBk29i0DaUS7EkqUcLb4IdcbjCUaBXcU9k-T59JJDuOkvsB_PDdKa0F3Q54_sAdIi_4tNK5oHYmT20OLV64mFLoiq2G2L-9ihkDdgJSg=w1280"/></center>

_________

**Integrantes del grupo**
- Argento, Lucas
- Espínola, Carla
- Gambarte, Antonella
- Putrino, Daniela
- Silvera, Ricardo
---------

#### **1. ¿Cuáles son los PEAS de este problema?**

- **Perfomance**: partiendo del estado inicial (cinco discos apilados de en varilla 1) moverlos a varilla 3, respetando las reglas del juego y resolviendo en la menor cantidad de operaciones / tiempo.

- **Enviroment**: cinco discos de distintos tamaños, base con tres varillas.

- **Actuators**: capacidad de mover los discos de una varilla a otra.

- **Sensors**: capacidad de detectar del estado actual de las varillas (discos en cada una y orden de esos discos).


#### **2. ¿Cuáles son las propiedades del entorno de trabajo?**

- **Totalmente observable**: el agente puede ver en todo momento el estado del entorno (posición de los discos en cada varilla).

- **Determinista**: el siguiente estado esta determinado por el estado actual y la acción que ejecuta el agente.

- **Secuencial**: la decisión actual puede afectar las decisiones futuras.

- **Estático**: el entorno no se modifica mientras el agente delibera.

- **Discreto**: tiene un número finito de estados distintos.

- **Agente individual**


#### **3. En el contexto de este problema, defina los siguientes conceptos:**

- **Estado**: Una configuración específica de los discos en las tres varillas, indicando en qué varilla está cada disco y su orden.
- **Espacio de estados**: El conjunto de todos los posibles estados alcanzables mediante movimientos válidos de los discos en las distintas varillas.
- **Árbol de búsqueda**: Representación de todos los estados posibles (nodos) y cómo llegamos a ellos (rama). Se recorren los caminos desde el estado inicias hasta llegar a al estado objetivo.
- **Nodo de búsqueda**: Configuración intermedia de los discos entre el estado inicial y el final representada por un nodo en el árbol de búsqueda. En este nodo se sabe cómo se llegó a este estado, es decir, cuáles fueron las configuraciones y acciones previas (camino de la rama), qué número de movimiento es el actual (profundidad) y cuál es el costo acumulado.
- **Objetivo**: Estado donde todos los discos están aplicados en la tercera varilla en forma de pirámide.
- **Acción**: Movimiento permitido (rama del árbol) de uno de los discos hacia alguna de las varillas que no es la actual mientras se siga con las reglas del juego.
- **Frontera**: Conjunto de nodos generados pero aún no explorados. Representa los posibles caminos a seguir en la búsqueda de la solución. Según el algoritmo que se use, puede ser una pila o una cola de estados a explorar.


#### **4. Implemente algun método de búsqueda**

En este caso se eligió implementar el método de costo uniforme. Para asignar el costo de cada camino se aplicó la fórmula:

_<center>costo_de_camino = 2^(numero_de_disco - 1)</center>_

donde el número del disco va, en este problema en particular, del 1 al 5, 1 para el más pequeño y 5 para el más grande, por lo que mover un disco más grande es más costoso. Cabe aclarar que no existe un costo diferente real para mover un disco chico o uno grande. Se asignan estos costos solo a los fines de implementar el algoritmo de Dijkstra, ya que si el costo para todos los caminos es el mismo, el algoritmo sería igual que al de búsqueda en anchura.

**Implementación:**


In [1]:
from aima_libs.hanoi_states import ProblemHanoi, StatesHanoi
from aima_libs.tree_hanoi import NodeHanoi
from queue import PriorityQueue

def calculate_cost(disk_number):
    move_cost = 2 ** (disk_number - 1)
    return move_cost

def uniform_cost_search(disks_amount):
    initial_disks = list(range(disks_amount, 0, -1))
    initial_state = StatesHanoi(initial_disks, [], [], max_disks=disks_amount)
    goal_state = StatesHanoi([], [], initial_disks, max_disks=disks_amount)
    problem = ProblemHanoi(initial=initial_state, goal=goal_state)
    #Para prioridad
    priority_queue = PriorityQueue()
    #Para guardar nodos visitados
    visited_nodes = {}
    # Raíz del árbol
    root = NodeHanoi(state=initial_state)
    priority_queue.put((0, root))

    while not priority_queue.empty():
        priority_value, new_node = priority_queue.get()
        frontier_nodes=new_node.expand(problem=problem)
        new_node.state.accumulated_cost = priority_value
        if problem.goal_test(new_node.state):
            last_node = new_node
            solution= last_node.solution()
            metrics = {
                "last_node": last_node,
                "max_depth": new_node.depth,
                "cost_total": new_node.state.accumulated_cost,
                "movements_total": len(solution),
            }
            
            return last_node, solution , metrics

        if new_node.state in visited_nodes and visited_nodes[new_node.state] <= priority_value:
            continue

        # Marcar el nodo como visitado con su costo acumulado
        visited_nodes[new_node.state] = priority_value

        for node in frontier_nodes:
            total_cost=priority_value + calculate_cost(node.action.disk)
            priority_queue.put((total_cost, node))
        
    
    print("No se encontró la solución")
    metrics = {
                "last_node": None,
                "max_depth": None,
                "cost_total": None,
            }
    return None, None , metrics


> ##### Solución y métricas:


In [2]:
last_node, solution, metrics= uniform_cost_search(5)

for act in solution:
    print(act)

print(metrics)

Move disk 1 from 1 to 3
Move disk 2 from 1 to 2
Move disk 1 from 3 to 2
Move disk 3 from 1 to 3
Move disk 1 from 2 to 1
Move disk 2 from 2 to 3
Move disk 1 from 1 to 3
Move disk 4 from 1 to 2
Move disk 1 from 3 to 2
Move disk 2 from 3 to 1
Move disk 1 from 2 to 1
Move disk 3 from 3 to 2
Move disk 1 from 1 to 3
Move disk 2 from 1 to 2
Move disk 1 from 3 to 2
Move disk 5 from 1 to 3
Move disk 1 from 2 to 1
Move disk 2 from 2 to 3
Move disk 1 from 1 to 3
Move disk 3 from 2 to 1
Move disk 1 from 3 to 2
Move disk 2 from 3 to 1
Move disk 1 from 2 to 1
Move disk 4 from 2 to 3
Move disk 1 from 1 to 3
Move disk 2 from 1 to 2
Move disk 1 from 3 to 2
Move disk 3 from 1 to 3
Move disk 1 from 2 to 1
Move disk 2 from 2 to 3
Move disk 1 from 1 to 3
{'last_node': <Node HanoiState:  |  | 5 4 3 2 1>, 'max_depth': 31, 'cost_total': 80, 'movements_total': 31}


#### **5. ¿Cuál es la complejidad teórica en tiempo y memoria del algoritmo elegido?**

- **Complejidad teórica en tiempo:** En el peor caso, la complejidad es O(b^d), donde b es el factor de ramificación (cantidad máxima de movimientos posibles desde un estado), d es la profundidad del nodo objetivo (cantidad de movimientos para llegar a la solución). Esto se debe a que el algoritmo puede llegar a expandir todos los nodos del espacio de estados antes de encontrar la solución óptima.

- **Complejidad teórica en memoria:** También es O(b^d), ya que debe almacenar todos los nodos generados en la frontera (cola de prioridad) y los visitados.

- **Aplicado a nuestro problema** el espacio de estados es O(3^n), ya que cada disco puede estar en cualquiera de las 3 varillas. Por lo tanto, tanto el tiempo como la memoria crecen exponencialmente con la cantidad de discos.


#### **6. A nivel de implementación, ¿cuánto tiempo y memoria utiliza el algoritmo? (Se recomienda ejecutarlo 10 veces y calcular el promedio y el desvío estándar de ambas métricas).**

Código para medición de tiempo de ejecución y uso de memoria:


In [3]:
%%timeit

## Tiempo de ejecución
last_node, solution, metrics= uniform_cost_search(5)

18.5 ms ± 328 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [4]:
import timeit
import math

def run_search():
    last_node, solution, metrics = uniform_cost_search(5)

number_of_runs = 10
times = []

for run in range(number_of_runs):
    time = timeit.timeit(run_search, number=1)
    print(f"Time spent on run {run}: {round(time,5)} seconds")
    times.append(time)

# Convert to ms
times_ms = [t * 1000 for t in times]

# avg and std dev
avg_time = sum(times_ms) / len(times_ms)
squared_diffs = [(t - avg_time) ** 2 for t in times_ms]
std_dev = math.sqrt(sum(squared_diffs) / len(times_ms))

print("-------------------------------------")
print(f"Tiempo promedio: {avg_time:.3f} ms")
print(f"Desvío estándar: {std_dev:.3f} ms")

Time spent on run 0: 0.02017 seconds
Time spent on run 1: 0.01896 seconds
Time spent on run 2: 0.01827 seconds
Time spent on run 3: 0.01869 seconds
Time spent on run 4: 0.01812 seconds
Time spent on run 5: 0.01846 seconds
Time spent on run 6: 0.01829 seconds
Time spent on run 7: 0.0182 seconds
Time spent on run 8: 0.01823 seconds
Time spent on run 9: 0.01825 seconds
-------------------------------------
Tiempo promedio: 18.562 ms
Desvío estándar: 0.588 ms


In [5]:
import tracemalloc

tracemalloc.start()
mem_usages = []

for run in range(number_of_runs):
    tracemalloc.start()
    run_search()
    _, memory_peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()

    memory_peak_mb = memory_peak / (1024 * 1024)  # bytes → MB
    print(f"Memoria pico en run {run}: {round(memory_peak_mb, 5)} MB")
    mem_usages.append(memory_peak_mb)

# Promedio y desvío estándar
avg_mem = sum(mem_usages) / len(mem_usages)
squared_diffs = [(m - avg_mem) ** 2 for m in mem_usages]
std_dev_mem = math.sqrt(sum(squared_diffs) / len(mem_usages))

print("-------------------------------------")
print(f"Memoria pico promedio: {avg_mem:.3f} MB")
print(f"Desvío estándar: {std_dev_mem:.3f} MB")

Memoria pico en run 0: 0.24709 MB
Memoria pico en run 1: 0.236 MB
Memoria pico en run 2: 0.23512 MB
Memoria pico en run 3: 0.23598 MB
Memoria pico en run 4: 0.23575 MB
Memoria pico en run 5: 0.23842 MB
Memoria pico en run 6: 0.23604 MB
Memoria pico en run 7: 0.23512 MB
Memoria pico en run 8: 0.23568 MB
Memoria pico en run 9: 0.23575 MB
-------------------------------------
Memoria pico promedio: 0.237 MB
Desvío estándar: 0.003 MB


**Luego de ejecutar el código 10 veces y realizar el cálculo del promedio y desvío estándar para cada métrica, se obtuvieron los siguientes resultados:**

> **Tiempo de ejecución**

>_Promedio:_ 18,562 ms  
> _Desvío estándar:_  0,588 ms

> **Memoria utilizada**

> _Promedio:_ 0,237 MB  
> _Desvío estándar:_ 0.003 MB


#### **7. Si la solución óptima es de 2k - 1 movimientos (siendo k el número de discos), ¿qué tan lejos está la solución encontrada por el algoritmo implementado de esa solución óptima? (Se recomienda ejecutar al menos 10 veces y usar el promedio de los trayectos obtenidos).**

In [6]:
disk_count = 5
optimal_solution = 2**disk_count - 1
print(f"Solución óptima para {disk_count} discos:", optimal_solution)

Solución óptima para 5 discos: 31


In [7]:
num_runs = 10
verbose = False
differences = []
for run in range(num_runs):
    last_node, solution, metrics = uniform_cost_search(disk_count)
    if last_node is not None:
        if verbose:
            print(f"Ejecutando búsqueda de costo uniforme para {disk_count} discos:")
            print(f"Profundidad máxima alcanzada: {metrics['max_depth']}")
            print(f"Diferencia en movimientos con la solucion optima: {optimal_solution - metrics['max_depth']}")
        difference = optimal_solution - metrics['max_depth']
        differences.append(difference)
    else:
        print("No se encontró solución.")

avg_dif = sum(differences) / len(differences)
std_dif = (sum((x - avg_dif) ** 2 for x in differences) / len(differences)) ** 0.5
print(f"Diferencia promedio en movimientos con la solución óptima: {avg_dif}")
print(f"Desviación estándar de la diferencia: {std_dif}")

Diferencia promedio en movimientos con la solución óptima: 0.0
Desviación estándar de la diferencia: 0.0


> **El algoritmo encuntra la solucion optima (31 movimientos). Esto no se modifica en las distintas ejecuciones, ya que es determinista y el estado inicial es siempre el mismo**

#### **NOTA**: A continuación se agrega el código para generar los jsons necesarios para el simulador.


In [7]:
last_node, solution, metrics= uniform_cost_search(5)
last_node.generate_solution_for_simulator()